In [7]:
from helper_functions import load_data_for_all_three
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_lg")
from tqdm import tqdm, tqdm_notebook

In [8]:
text_data, labels = load_data_for_all_three('data')
df = pd.DataFrame(list(zip(text_data, labels)), columns=['text_data', 'label'])

Processing file: fitzgerald_all_the_sad.txt
Processing file: fitzgerald_bablyon_revisited.txt
Processing file: fitzgerald_beautiful_and_damned.txt
Processing file: fitzgerald_flappers_and_philosophers.txt
Processing file: fitzgerald_gatsby.txt
Processing file: fitzgerald_tales_jazz_age.txt
Processing file: fitzgerald_tender_is.txt
Processing file: fitzgerald_the_vegtable.txt
Processing file: fitzgerald_this_side.txt
Processing file: hemingway_across_the_river.txt
Processing file: hemingway_bell_tolls.txt
Processing file: hemingway_collected_stories.txt
Processing file: hemingway_farewell.txt
Processing file: hemingway_green_hills_africa.txt
Processing file: hemingway_in_our_time.txt
Processing file: hemingway_men_without_women.txt
Processing file: hemingway_old_man.txt
Processing file: hemingway_sun_also.txt
Processing file: hemingway_ten_poems.txt
Processing file: hemingway_winner_take_nothing.txt
Processing file: steinbeck_cannery_row.txt
Processing file: steinbeck_east_of_eden.txt
P

In [13]:
nlp.max_length = 2000000
# define a function to extract POS tags from a text and return a dictionary with the counts of each tag
def get_pos_counts(text):
    doc = nlp(text)
    pos_counts = {}
    for token in doc:
        pos = token.pos_
        if pos in pos_counts:
            pos_counts[pos] += 1
        else:
            pos_counts[pos] = 1
    return pos_counts

# apply the get_pos_counts function to each row in the dataframe
#df["pos_counts"] = df["text_data"].progress_apply(get_pos_counts)

def get_pos_counts_wrapper(text):
    return get_pos_counts(text)

df["pos_counts"] = list(tqdm(df["text_data"].apply(get_pos_counts_wrapper)))

# extract the POS tags and their counts into separate columns in the dataframe
pos_tags = set.union(*[set(d.keys()) for d in df["pos_counts"]])
for tag in pos_tags:
    df[tag] = df["pos_counts"].apply(lambda x: x.get(tag, 0))

df.head()